In [2]:
import os
import re
import pandas as pd
from io import StringIO

In [139]:
import numpy as np

In [6]:
import os

In [92]:
import difflib

In [17]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

c:\Users\samtg\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [109]:
import unicodedata

In [7]:
data_folder_relative = os.path.join('..', '..', 'data', 'raw', 'lissy', 'france_gini.txt')
data_folder_absolute = os.path.abspath(data_folder_relative)

In [11]:
data_cleaned_rel = os.path.join('..', '..', 'data', 'cleaned', 'france_party.csv')
data_cleaned = os.path.abspath(data_cleaned_rel)

In [8]:
df = pd.read_fwf(data_folder_absolute, skiprows=3, header=None, names=["region", "year", "avg_gini"])

In [129]:
df.to_csv('data/cleaned/lissy/france_gini.csv', index=False)

In [68]:
france = pd.read_csv(data_cleaned)

C:\Users\samtg\AppData\Local\Temp\ipykernel_17704\2099512860.py:1: DtypeWarning: Columns (10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  france = pd.read_csv(data_cleaned)


In [70]:
france['year'] = france['elec_year']

In [71]:
join = df.merge(france, on=['region','year'] , how='inner')

In [116]:
df['cleaned_region'] = df['region'].str.lower().str.replace('[^\w\s]', '')
france['cleaned_region'] = france['region'].str.lower().str.replace('[^\w\s]', '')

In [117]:
df['cleaned_region'] = df['cleaned_region'].str.strip()
france['cleaned_region'] = france['cleaned_region'].str.strip()


In [118]:
matched_regions = {}

In [124]:
for year in df['year'].unique():
    # Filter DataFrames by the current year
    df_year = df[df['year'] == year]
    france_year = france[france['year'] == year]
    
    # Iterate through unique cleaned regions in df for the current year
    for cleaned_region in df_year['cleaned_region'].unique():
        # Find the best match in France DataFrame using difflib
        matches = difflib.get_close_matches(cleaned_region, france_year['cleaned_region'], n=1, cutoff=0.4)
        
        # Check if a valid match is found
        if matches:
            # Get the best match
            matched_region = matches[0]
            
            # Store the matched region and year
            matched_regions[(year, cleaned_region)] = (year, matched_region)

In [129]:
matched_regions

{(1998, 'alsace'): (1998, 'alsace'),
 (1998, 'aquitaine'): (1998, 'aquitaine'),
 (1998, 'auvergne'): (1998, 'auvergne'),
 (1998, 'basse-normandie'): (1998, 'basse normandie'),
 (1998, 'bourgogne'): (1998, 'bourgogne'),
 (1998, 'bretagne'): (1998, 'bretagne'),
 (1998, 'centre'): (1998, 'centre'),
 (1998, 'champagne-arden'): (1998, 'champagne ardenne'),
 (1998, 'franche-comté'): (1998, 'france comte'),
 (1998, 'haute-normandie'): (1998, 'haute normandie'),
 (1998, 'ile-de-france'): (1998, 'ile de france'),
 (1998, 'languedoc-rouss'): (1998, 'languedoc roussillon'),
 (1998, 'limousin'): (1998, 'limousin'),
 (1998, 'lorraine'): (1998, 'lorraine'),
 (1998, 'midi-pyrénées'): (1998, 'midi pyrenees'),
 (1998, 'nord-pas de cal'): (1998, 'nord pas de calais'),
 (1998, 'pays de la loir'): (1998, 'pays de la loire'),
 (1998, 'picardie'): (1998, 'picardie'),
 (1998, 'poitou-charente'): (1998, 'poitou charente'),
 (1998, 'rhône-alpes'): (1998, 'rhone alpes'),
 (2004, 'alsace'): (2004, 'alsace'),
 (2

In [125]:
matched_region

'auvergne - rhône-alpes'

In [126]:
matched_df = pd.DataFrame(matched_regions.values(), columns=['year', 'matched_region'])

# Merge the DataFrames on 'year' and 'matched_region'
merged_df = pd.merge(df, matched_df, left_on=['year', 'cleaned_region'], right_on=['year', 'matched_region'], how='left')

In [127]:
merged_df

,region,year,avg_gini,similarity_ratio,cleaned_region,matched_region
0,Alsace,1997,0.286,0,alsace,NaN
1,Alsace,1998,0.273,0,alsace,alsace
2,Alsace,1999,0.289,0,alsace,NaN
3,Alsace,2000,0.299,0,alsace,NaN
4,Alsace,2001,0.304,0,alsace,NaN
...,...,...,...,...,...,...
497,Rhône-Alpes,2014,0.319,1,rhône-alpes,NaN
498,Rhône-Alpes,2015,0.329,1,rhône-alpes,NaN
499,Rhône-Alpes,2016,0.322,1,rhône-alpes,NaN
500,Rhône-Alpes,2017,0.308,1,rhône-alpes,NaN


In [122]:
# Merge with an inner join based on 'region' and 'year'
join = merged_df.merge(france, on=['region', 'year'], how='inner')


In [123]:
join

,region,year,avg_gini,similarity_ratio,cleaned_region_x,matched_region,Unnamed: 0,country,country_abbrev_x,elec_year,electorate,valid,votes,party_abbrev,vote_count,abbreviation,notes,party,country_abbrev_y,cleaned_region_y
0,Alsace,1998,0.273,0,alsace,alsace,44,France,FRA,1998,1128552.0,578651.0,612189.0,AHS,NaN,NaN,NaN,NaN,NaN,alsace
1,Alsace,1998,0.273,0,alsace,alsace,225,France,FRA,1998,1128552.0,578651.0,612189.0,ALNO,NaN,NaN,NaN,NaN,NaN,alsace
2,Alsace,1998,0.273,0,alsace,alsace,406,France,FRA,1998,1128552.0,578651.0,612189.0,ASOK,NaN,NaN,NaN,NaN,NaN,alsace
3,Alsace,1998,0.273,0,alsace,alsace,587,France,FRA,1998,1128552.0,578651.0,612189.0,BCH,NaN,NaN,NaN,NaN,NaN,alsace
4,Alsace,1998,0.273,0,alsace,alsace,768,France,FRA,1998,1128552.0,578651.0,612189.0,BGO,NaN,NaN,NaN,NaN,NaN,alsace
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113203,Picardie,2010,0.294,0,picardie,picardie,386237,France,FRA,2010,1320977.0,643516.0,675704.0,WCP,NaN,NaN,NaN,NaN,NaN,picardie
113204,Picardie,2010,0.294,0,picardie,picardie,386361,France,FRA,2010,1321513.0,574494.0,602227.0,WEP,NaN,NaN,NaN,NaN,NaN,picardie
113205,Picardie,2010,0.294,0,picardie,picardie,386418,France,FRA,2010,1320977.0,643516.0,675704.0,WEP,NaN,NaN,NaN,NaN,NaN,picardie
113206,Picardie,2010,0.294,0,picardie,picardie,386542,France,FRA,2010,1321513.0,574494.0,602227.0,WP,NaN,NaN,NaN,NaN,NaN,picardie


In [115]:
merged_df

,region,year,avg_gini,similarity_ratio,cleaned_region,matched_region
0,Alsace,1997,0.286,0,Alsace,NaN
1,Alsace,1998,0.273,0,Alsace,Alsace
2,Alsace,1999,0.289,0,Alsace,NaN
3,Alsace,2000,0.299,0,Alsace,NaN
4,Alsace,2001,0.304,0,Alsace,NaN
...,...,...,...,...,...,...
494,Rhône-Alpes,2014,0.319,1,Rhone-Alpes,NaN
495,Rhône-Alpes,2015,0.329,1,Rhone-Alpes,NaN
496,Rhône-Alpes,2016,0.322,1,Rhone-Alpes,NaN
497,Rhône-Alpes,2017,0.308,1,Rhone-Alpes,NaN


In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [134]:
from sklearn.metrics.pairwise import cosine_similarity

In [135]:
tfidf_vectorizer = TfidfVectorizer()
X_df = tfidf_vectorizer.fit_transform(df['cleaned_region'])
X_france = tfidf_vectorizer.transform(france['cleaned_region'])

In [136]:
similarity_matrix = cosine_similarity(X_france, X_df)

In [137]:
similarity_threshold = 0.7

In [146]:
predicted_matches = []
confidence_scores = []

In [147]:
for i in range(len(france)):
    best_match_index = np.argmax(similarity_matrix[i])
    best_similarity_score = similarity_matrix[i][best_match_index]
    
    if best_similarity_score > similarity_threshold:
        predicted_match = df.iloc[best_match_index]['region']
    else:
        predicted_match = None
    
    predicted_matches.append(predicted_match)
    confidence_scores.append(best_similarity_score)

In [148]:
france['predicted_region'] = predicted_matches
france['confidence_score'] = confidence_scores

Gonna try and do the same with national data